In [1]:
import logging
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_json
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType, MapType
from define_schema.tweets import TweetDocument
def create_spark_connection():
    try:
        spark_conn = SparkSession.builder \
            .appName('KafkaToHDFS') \
            .config('spark.hadoop.hadoop.security.authentication', 'simple') \
            .config("spark.hadoop.dfs.replication", "1") \
            .config('spark.jars.packages', "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.3") \
            .getOrCreate()

        spark_conn.sparkContext.setLogLevel("ERROR")
        logging.info("Spark connection created successfully!")
        print("Connected to HDFS successfully")
        return spark_conn
    except Exception as e:
        logging.error(f"Couldn't create the spark session due to exception: {e}")
        return None
    
spark_conn = create_spark_connection()

your 131072x1 screen size is bogus. expect trouble
24/12/08 16:43:35 WARN Utils: Your hostname, DESKTOP-H2N322M resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/12/08 16:43:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/mnt/c/Users/bienn/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/bien/.ivy2/cache
The jars for the packages stored in: /home/bien/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2b20ad91-6c2c-405c-ae2f-f511de626a2f;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.3 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.3 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in local-m2-cache
	found org.xerial.snappy#snappy-java;1.1.10.5 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in local-m2-cache
	found org.apache.hadoop#hadoop-client-api;3.3.4 in local-m2-cache
	found commons-logging#commons-logging;1.1.3 in local-m2-cache
	found com.google.code.findbugs#jsr305;3.0.0 in local-m2-cache
	found org.apache.commons#commons-pool2;2.11.1 in central
:: resolution report :: resolve

Connected to HDFS successfully


In [ ]:
df = spark.read.format("json").option("multiline", "true").load("hdfs://namenode:9000/user/hdfs/tweets/*.json")
df.printSchema()

ERROR:root:KeyboardInterrupt while sending command.                 (0 + 2) / 2]
Traceback (most recent call last):
  File "/home/bien/.pyenv/versions/3.8.10/lib/python3.8/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/bien/.pyenv/versions/3.8.10/lib/python3.8/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/home/bien/.pyenv/versions/3.8.10/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

24/12/08 16:48:12 ERROR Executor: Exception in task 0.0 in stage 0.0 (TID 0)/ 2]
org.apache.hadoop.hdfs.BlockMissingException: Could not obtain block: BP-1871902167-172.18.0.5-1733648584451:blk_1073741983_1159 file=/user/hdfs/tweets/part-00000-55e3110e-dd11-4a92-b5f3-f6948311dacf-c000.json No live nodes contain current block Block locations: DatanodeInfoWithStorage[172.18.0.3:9866,DS-89eefc00-231f-4bf0-a364-57f541fa701a,DISK] Dead nodes:  DatanodeInfoWithStorage[172.18.0.3:9866,DS-89eefc00-231f-4bf0-a364-57f541fa701a,DISK]
	at org.apache.hadoop.hdfs.DFSInputStream.refetchLocations(DFSInputStream.java:1007)
	at org.apache.hadoop.hdfs.DFSInputStream.chooseDataNode(DFSInputStream.java:990)
	at org.apache.hadoop.hdfs.DFSInputStream.chooseDataNode(DFSInputStream.java:969)
	at org.apache.hadoop.hdfs.DFSInputStream.blockSeekTo(DFSInputStream.java:677)
	at org.apache.hadoop.hdfs.DFSInputStream.readWithStrategy(DFSInputStream.java:884)
	at org.apache.hadoop.hdfs.DFSInputStream.read(DFSInputStre

In [ ]:
def connect_to_kafka(spark_conn):
    try:
        kafka_df = spark_conn.readStream \
            .format("kafka") \
            .option("kafka.bootstrap.servers", "broker:9092") \
            .option("subscribe", "tweets_topic") \
            .option("multiline", "true")\
            .load()
        logging.info("Kafka DataFrame created successfully")
        print("Kafka DataFrame created successfully")

        return kafka_df
    except Exception as e:
        logging.error(f"Kafka DataFrame could not be created: {e}", exc_info=True)
        return None
kafka_df = connect_to_kafka(spark_conn)

Kafka DataFrame created successfully


In [ ]:
def write_to_hdfs(df):
    try:
        streaming_query = df.writeStream \
            .outputMode("append") \
            .format("parquet") \
            .option("path", "hdfs://localhost:9000/user/hdfs/tweets/") \
            .option("checkpointLocation", "hdfs://localhost:9000/user/hdfs/tweets_checkpoint/") \
            .start()

        logging.info("Writing data to HDFS")
        print("Writing data to HDFS")
        streaming_query.awaitTermination()
    except Exception as e:
        logging.error(f"Failed to write to HDFS: {e}", exc_info=True)
        print("Failed to write to HDFS")

tweet_schema = TweetDocument().get_schema()
kafka_df = kafka_df.select(col("value").cast("string").alias("json_data"))
parsed_df = kafka_df.select(from_json(col("json_data"), tweet_schema).alias("data")).select("data.*")

write_to_hdfs(parsed_df)

In [ ]:
df = spark.read.format("json").load("hdfs://namenode:9000/user/hdfs/tweets/*.json")
df.write.format("json").save("files")

root
 |-- _id: string (nullable = true)
 |-- author: string (nullable = true)
 |-- authorName: string (nullable = true)
 |-- created: string (nullable = true)
 |-- hashTags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- likes: long (nullable = true)
 |-- replyCounts: long (nullable = true)
 |-- retweetCounts: long (nullable = true)
 |-- text: string (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- url: string (nullable = true)
 |-- userMentions: struct (nullable = true)
 |    |-- 1499055417229561859: string (nullable = true)
 |-- views: long (nullable = true)

